In [1]:
import boto3
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import sys
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
import json

sys.path.append('..')
import CustomHyperModelImages
import EnergyPricesLibrary as Ep

from kerastuner.tuners import BayesianOptimization

%load_ext autoreload
%autoreload 2

In [2]:
def make_predictions(model,scaler_y,trainX,trainY,testX,testY,n_steps_out,len_output_features):
    
    # make predictions
    trainPredict = model.predict(trainX)
    trainPredict = trainPredict.reshape(trainPredict.shape[0]*n_steps_out,len_output_features)
    testPredict  = model.predict(testX)
    testPredict  = testPredict.reshape(testPredict.shape[0]*n_steps_out,len_output_features)
    
    # invert predictions
    trainPredict = scaler_y.inverse_transform(trainPredict)
    trainY_ = scaler_y.inverse_transform(trainY.reshape(trainY.shape[0]*n_steps_out,len_output_features))
    
    testPredict = scaler_y.inverse_transform(testPredict)
    testY_ = scaler_y.inverse_transform(testY.reshape(testY.shape[0]*n_steps_out,len_output_features))
        
    return trainPredict,trainY_,testPredict,testY_

def get_metrics(trainY,trainPredict,testY,testPredict):
    
    trainMAPE  = Ep.MAPE(trainPredict,trainY)
    testMAPE  = Ep.MAPE(testPredict,testY)
    
    train_sMAPE  = Ep.sMAPE(trainY,trainPredict)
    test_sMAPE  = Ep.sMAPE(testY,testPredict)
    
    return trainMAPE,testMAPE,train_sMAPE,test_sMAPE

In [3]:
def get_dataset_images(lista_rutas,lista_fechas,n_steps_in,overlap,day,
                       start_date_train,start_date_val,start_date_test,end_date_test,
                       n_steps_out,output_columns):
    
    dataset_df = pd.DataFrame(lista_rutas,index=lista_fechas,columns=['Precipitacion','Temperatura'])
    inputs_columns=dataset_df.columns.values
    
    dataset_df['day_of_week'] = dataset_df.index.day_name()
    
    len_output_features = len(output_columns)

    IMG_HEIGHT,IMG_WIDTH = 128,128

    results = Ep.SplitTimeseriesMultipleTimesBackAhead_DifferentTimes_Images(df_x=dataset_df,df_y=precio_bolsa,
                                                                             day=day,
                                                                             start_date_train=start_date_train,
                                                                             start_date_val=start_date_val,
                                                                             start_date_test=start_date_test,
                                                                             end_date_test=end_date_test,
                                                                             n_steps_out=n_steps_out,
                                                                             n_steps_in=n_steps_in,overlap=overlap,
                                                                             output_features=output_columns,
                                                                             input_features=inputs_columns,
                                                                             IMG_HEIGHT=IMG_HEIGHT,IMG_WIDTH=IMG_WIDTH)

    return results

In [4]:
def crear_callbacks():
    
    callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                              factor=0.1,
                                                              min_lr=1e-5,
                                                              patience=5,
                                                              verbose=1)

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=10,
                                                      mode='min')

    callbacks = [callback_reduce_lr,early_stopping]
    
    return callbacks

In [5]:
def generar_arquitecturas(input_shape,n_steps_out):

    arquitectura13 = CustomHyperModelImages.ArquitecturaI13(input_shape=input_shape,n_steps_out=n_steps_out)
    arquitectura14 = CustomHyperModelImages.ArquitecturaI14(input_shape=input_shape,n_steps_out=n_steps_out)

    arq_list = [arquitectura13,arquitectura14]
    
    return arq_list

# DATASET

In [6]:
climatic_images_prcp_dir = os.path.join('..','..','..','dataset','Climatic Images','PRCP')

In [7]:
climatic_images_tavg_dir = os.path.join('..','..','..','dataset','Climatic Images','TAVG')

In [8]:
precio_bolsa_path = os.path.join('..','..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Precio_Bolsa.xlsx')
precio_bolsa = pd.read_excel(precio_bolsa_path)
precio_bolsa = precio_bolsa.set_index('Fecha')

In [9]:
precio_bolsa['day_of_week'] = precio_bolsa.index.day_name()

## Build Window

In [10]:
lista_fechas = list()
lista_rutas = list()
for prcp_file,tavg_file in zip(os.listdir(climatic_images_prcp_dir),os.listdir(climatic_images_tavg_dir)):
    fecha = prcp_file.split('.')[0]
    ruta_prcp = os.path.join(climatic_images_prcp_dir,prcp_file)
    ruta_tavg = os.path.join(climatic_images_tavg_dir,tavg_file)
    lista_fechas.append(pd.to_datetime(fecha))
    lista_rutas.append([ruta_prcp,ruta_tavg])

In [11]:
Days = np.array(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

start_date_train = '2000-02-01'
start_date_val = '2020-01-01'
start_date_test = '2020-04-01'
end_date_test = '2020-05-01'
n_steps_out=24
output_columns = ['$kWh']
len_output_features = len(output_columns)
n_steps_in = 2
overlap = 1

In [12]:
dict_days_test = dict()

for j,d in enumerate(Days):
    
    result_from_images = get_dataset_images(lista_rutas,lista_fechas,n_steps_in=n_steps_in,overlap=overlap,day=d,
                                            start_date_train=start_date_train,start_date_val=start_date_val,
                                            start_date_test=start_date_test,end_date_test=end_date_test,
                                            n_steps_out=n_steps_out,output_columns=output_columns)
    
    trainX_I,trainY_I,valX_I,valY_I,testX_I,testY_I,scaler_y_I,dataset_x_I,dataset_y_I = result_from_images
    
    images_input_shape = trainX_I[0].shape
    callbacks = crear_callbacks()
    arq_list = generar_arquitecturas(images_input_shape,n_steps_out)
    
    arq_idx = 13
    arq_best_models = dict()

    for arq in arq_list:
        
        project_name = '{}-Arquitectura{}'.format(d,arq_idx)

        bayesian_tuner = BayesianOptimization(
            arq,
            objective='val_loss',
            num_initial_points=1,
            max_trials=10,
            directory=os.path.normpath('C:/my_dir'),
            project_name=project_name
        )

        # Overview of the task
        bayesian_tuner.search_space_summary()

        # Performs the hyperparameter tuning
        search_start = time.time()
        bayesian_tuner.search(x=trainX_I,y=trainY_I,
                          epochs=200,
                          validation_data=(valX_I,valY_I),
                          callbacks=callbacks)
        search_end = time.time()
        elapsed_time = search_end - search_start

        dict_key = project_name

        arq_best_models[dict_key] = dict()
        bs_model = bayesian_tuner.oracle.get_best_trials(1)[0]

        model = bayesian_tuner.get_best_models(num_models=1)[0]

        trainPredict,trainY_true,testPredict,testY_true = make_predictions(model,scaler_y_I,trainX_I,trainY_I,valX_I,valY_I,
                                                                           n_steps_out,len_output_features)

        trainMAPE,testMAPE,train_sMAPE,test_sMAPE = get_metrics(trainY_true,trainPredict,testY_true,testPredict)

        arq_best_models[dict_key]['Score'] = bs_model.score
        arq_best_models[dict_key]['Tiempo Scaneo'] = elapsed_time
        arq_best_models[dict_key]['Mape Train'] = trainMAPE
        arq_best_models[dict_key]['Mape Test'] = testMAPE
        arq_best_models[dict_key]['sMape Train'] = train_sMAPE
        arq_best_models[dict_key]['sMape Test'] = test_sMAPE

        if bs_model.hyperparameters.values:
            for hp, value in bs_model.hyperparameters.values.items():
                arq_best_models[dict_key][hp] = value

        arq_idx += 1
        
    with open('{}-BestModels.json'.format(d), 'w') as outfile:
        json.dump(arq_best_models, outfile)

Trial 10 Complete [00h 00m 18s]
val_loss: 3651.11279296875

Best val_loss So Far: 736.3565673828125
Total elapsed time: 00h 15m 03s
INFO:tensorflow:Oracle triggered exit
